# Artificial Intelligence Nanodegree
## Machine Translation Project
In this notebook, sections that end with **'(IMPLEMENTATION)'** in the header indicate that the following blocks of code will require additional functionality which you must provide. Please be sure to read the instructions carefully!

## Introduction
In this notebook, you will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Your completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - You'll convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, to design your own model!
- **Prediction** Run the model on English text.

## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a dataset we created for this project that contains a small vocabulary.  You'll be able to train your model in a reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [1]:
import helper


# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [2]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  This should save you some time, but the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [3]:
import collections


english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
For this project, you won't use text data as input to your model. Instead, you'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

Time to start preprocessing the data...
### Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Use this function to tokenize `english_sentences` and `french_sentences` in the cell below.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [4]:
import project_tests as tests
from keras.preprocessing.text import Tokenizer


def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # TODO: Implement

    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(x)
    #tokenizer.texts_to_sequences(x)
    return list(tokenizer.texts_to_sequences(x)), tokenizer

tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

Using TensorFlow backend.


{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [5]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences


def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # TODO: Implement
    return pad_sequences(x, maxlen=length, padding = 'post')

tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


### Preprocess Pipeline
Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline.  Instead, we've provided you with the implementation of the `preprocess` function.

In [9]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)

print('Data Preprocessed')

Data Preprocessed


## Models
In this section, you will experiment with various neural network architectures.
You will begin by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an optional Encoder-Decoder RNN

After experimenting with the four simple architectures, you will construct a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation.  You'll be using this function to better understand the output of the neural network.

In [10]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### Model 1: RNN (IMPLEMENTATION)
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, you'll build a RNN that translates English to French.

In [9]:
from keras.layers import GRU, Input, Dense, TimeDistributed
from keras.models import Model, Sequential
from keras.layers import Activation, LSTM
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy


def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Build the layers
    model = Sequential()
    model.add(LSTM(32, input_shape=(input_shape[1], 1), return_sequences=True))
    model.add(Dense(french_vocab_size, activation='softmax'))
    learning_rate= 0.01
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model

tests.test_simple_model(simple_model)


# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)
simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=30, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/30
110288/110288 [==============================] - 8s - loss: 2.6526 - acc: 0.4604 - val_loss: 2.1237 - val_acc: 0.5112
Epoch 2/30
110288/110288 [==============================] - 7s - loss: 1.8715 - acc: 0.5544 - val_loss: 1.6810 - val_acc: 0.5759
Epoch 3/30
110288/110288 [==============================] - 7s - loss: 1.5982 - acc: 0.5923 - val_loss: 1.5329 - val_acc: 0.6025
Epoch 4/30
110288/110288 [==============================] - 7s - loss: 1.4858 - acc: 0.6112 - val_loss: 1.4398 - val_acc: 0.6143
Epoch 5/30
110288/110288 [==============================] - 7s - loss: 1.4061 - acc: 0.6230 - val_loss: 1.3728 - val_acc: 0.6335
Epoch 6/30
110288/110288 [==============================] - 7s - loss: 1.3485 - acc: 0.6336 - val_loss: 1.3221 - val_acc: 0.6401
Epoch 7/30
110288/110288 [==============================] - 7s - loss: 1.3052 - acc: 0.6406 - val_loss: 1.2863 - val_acc: 0.6408
Epoch 8/30
110288/110288 [====================

### Model 2: Embedding (IMPLEMENTATION)
![RNN](images/embedding.png)
You've turned the words into ids, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, you'll create a RNN model using embedding.

In [10]:
from keras.layers.embeddings import Embedding


def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement

    model = Sequential()
    model.add(Embedding(english_vocab_size, 64, input_length=input_shape[1]))
    model.add(LSTM(32, return_sequences=True))
    model.add(Dense(french_vocab_size, activation='softmax'))
    
    learning_rate= 0.01
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

tests.test_embed_model(embed_model)

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
#tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
embed_rnn_model = embed_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)

embed_rnn_model.summary()
embed_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=30, validation_split=0.2)

print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))
# TODO: Reshape the input

# TODO: Train the neural network

# TODO: Print prediction(s)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 21, 64)            12800     
_________________________________________________________________
lstm_4 (LSTM)                (None, 21, 32)            12416     
_________________________________________________________________
dense_4 (Dense)              (None, 21, 345)           11385     
Total params: 36,601
Trainable params: 36,601
Non-trainable params: 0
_________________________________________________________________
Train on 110288 samples, validate on 27573 samples
Epoch 1/30
110288/110288 [==============================] - 8s - loss: 2.9054 - acc: 0.4530 - val_loss: 1.8243 - val_acc: 0.5616 loss: 3.7832 - a - ETA: 4s - los - ETA: 3s - loss: 3.3611 - acc: 0.4 - ETA: 2s - loss: 3.3263 - - ETA: 1s -
Epoch 2/30
110288/110288 [==============================] - 7s - loss: 1.3742 - acc: 0.6555 - val_loss: 1.0552 - v

### Model 3: Bidirectional RNNs (IMPLEMENTATION)
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [12]:
from keras.layers import GRU, Input, Dense, TimeDistributed
from keras.models import Model, Sequential
from keras.layers import Activation, LSTM
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
from keras.layers import Bidirectional


def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    model = Sequential()
    model.add(Bidirectional(LSTM(32, return_sequences=True),input_shape=(input_shape[1], 1)))
    model.add(Dense(french_vocab_size, activation='softmax'))
    
    learning_rate= 0.01
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model
tests.test_bd_model(bd_model)

# TODO: Train and Print prediction(s)

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
bd_rnn_model = bd_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)
bd_rnn_model.summary()
bd_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=30, validation_split=0.2)

#Print prediction(s)
print(logits_to_text(bd_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

# TODO: Train and Print prediction(s)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_5 (Bidirection (None, 21, 64)            8704      
_________________________________________________________________
dense_5 (Dense)              (None, 21, 345)           22425     
Total params: 31,129
Trainable params: 31,129
Non-trainable params: 0
_________________________________________________________________
Train on 110288 samples, validate on 27573 samples
Epoch 1/30
110288/110288 [==============================] - 11s - loss: 2.3119 - acc: 0.5195 - val_loss: 1.6129 - val_acc: 0.5913
Epoch 2/30
110288/110288 [==============================] - 11s - loss: 1.4589 - acc: 0.6176 - val_loss: 1.3578 - val_acc: 0.6364
Epoch 3/30
110288/110288 [==============================] - 11s - loss: 1.3095 - acc: 0.6433 - val_loss: 1.2654 - val_acc: 0.6516
Epoch 4/30
110288/110288 [==============================] - 11s - loss: 1.2326 - acc: 0.6565 

### Model 4: Encoder-Decoder (OPTIONAL)
Time to look at encoder-decoder models.  This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

Create an encoder-decoder model in the cell below.

In [14]:
from keras.layers import RepeatVector


def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # OPTIONAL: Implement    
    model = Sequential()
    model.add(LSTM(32, input_shape=(input_shape[1], 1), return_sequences=False))
    model.add(RepeatVector(output_sequence_length))
    model.add(LSTM(32, return_sequences=True))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax')))
    
    learning_rate= 0.01
    model.compile(loss=sparse_categorical_crossentropy, optimizer=Adam(learning_rate), 
                  metrics=['accuracy'] )
    
    return model

# OPTIONAL: Train and Print prediction(s)

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
encdec_rnn_model = encdec_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)
encdec_rnn_model.summary()
encdec_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=30, validation_split=0.2)

#Print prediction(s)
print(logits_to_text(encdec_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

# OPTIONAL: Train and Print prediction(s)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 32)                4352      
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 21, 32)            0         
_________________________________________________________________
lstm_9 (LSTM)                (None, 21, 32)            8320      
_________________________________________________________________
time_distributed_2 (TimeDist (None, 21, 345)           11385     
Total params: 24,057
Trainable params: 24,057
Non-trainable params: 0
_________________________________________________________________
Train on 110288 samples, validate on 27573 samples
Epoch 1/30
110288/110288 [==============================] - 11s - loss: 3.0945 - acc: 0.4220 - val_loss: 2.6173 - val_acc: 0.4423
Epoch 2/30
110288/110288 [==============================] - 10s - loss: 2.5582 - acc: 0.4

### Model 5: Custom (IMPLEMENTATION)
Use everything you learned from the previous models to create a model that incorporates embedding and a bidirectional rnn into one model.

## Prediction (IMPLEMENTATION)

In [15]:
from keras.layers import GRU, Input, Dense, TimeDistributed
from keras.models import Model, Sequential
from keras.layers import Activation, LSTM
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
from keras.layers import RepeatVector, Bidirectional
from keras.layers.embeddings import Embedding
from keras.callbacks import ModelCheckpoint 


def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    model = Sequential()
    model.add(Embedding(english_vocab_size, 64, input_length=input_shape[1]))
    model.add(LSTM(32, return_sequences=False))
    model.add(RepeatVector(output_sequence_length))
    model.add(Bidirectional(LSTM(32, return_sequences=True)))
    #model.add(LSTM(32, return_sequences=True))
    model.add(TimeDistributed(Dense(32, activation='relu')))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax')))
    model.summary()

    
    learning_rate= 0.01
    model.compile(loss=sparse_categorical_crossentropy, optimizer=Adam(learning_rate), 
                  metrics=['accuracy'] )
    
    return model
    
tests.test_model_final(model_final)

print('Final Model Loaded')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 15, 64)            12736     
_________________________________________________________________
lstm_10 (LSTM)               (None, 32)                12416     
_________________________________________________________________
repeat_vector_3 (RepeatVecto (None, 21, 32)            0         
_________________________________________________________________
bidirectional_6 (Bidirection (None, 21, 64)            16640     
_________________________________________________________________
time_distributed_3 (TimeDist (None, 21, 32)            2080      
_________________________________________________________________
time_distributed_4 (TimeDist (None, 21, 344)           11352     
Total params: 55,224
Trainable params: 55,224
Non-trainable params: 0
_________________________________________________________________
Final 

In [18]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import ModelCheckpoint 

def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # TODO: Train neural network using model_final
    model = None
    tmp_x = x
    # Train the neural network
    model = model_final(
        tmp_x.shape,
        y.shape[1],
        len(x_tk.word_index)+1,
        len(y_tk.word_index)+1)
    
    checkpointer = ModelCheckpoint(filepath='model.weights.best_no1.hdf5', verbose=1, 
                               save_best_only=True)

    model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=200, validation_split=0.2,
              callbacks=[checkpointer])
    #print(logits_to_text(model.predict(tmp_x[:1])[0], french_tokenizer))
    model.save('model_save.hdf5')
    
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.argmax(x)] for x in y[0]]))


final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 15, 64)            12800     
_________________________________________________________________
lstm_17 (LSTM)               (None, 32)                12416     
_________________________________________________________________
repeat_vector_9 (RepeatVecto (None, 21, 32)            0         
_________________________________________________________________
bidirectional_7 (Bidirection (None, 21, 64)            16640     
_________________________________________________________________
time_distributed_17 (TimeDis (None, 21, 32)            2080      
_________________________________________________________________
time_distributed_18 (TimeDis (None, 21, 345)           11385     
Total params: 55,321
Trainable params: 55,321
Non-trainable params: 0
_________________________________________________________________
Train 

110288/110288 [==============================] - 14s - loss: 0.3785 - acc: 0.8902 - val_loss: 0.3922 - val_acc: 0.8878
Epoch 50/200
110288/110288 [==============================] - 14s - loss: 0.3736 - acc: 0.8921 - val_loss: 0.4315 - val_acc: 0.8772
Epoch 51/200
110288/110288 [==============================] - 14s - loss: 0.3701 - acc: 0.8936 - val_loss: 0.3784 - val_acc: 0.8928
Epoch 52/200
110288/110288 [==============================] - 14s - loss: 0.3648 - acc: 0.8956 - val_loss: 0.3720 - val_acc: 0.8965
Epoch 53/200
110288/110288 [==============================] - 14s - loss: 0.3622 - acc: 0.8970 - val_loss: 0.3678 - val_acc: 0.8985
Epoch 54/200
110288/110288 [==============================] - 14s - loss: 0.3619 - acc: 0.8975 - val_loss: 0.3654 - val_acc: 0.8985
Epoch 55/200
110288/110288 [==============================] - 14s - loss: 0.3487 - acc: 0.9025 - val_loss: 0.3589 - val_acc: 0.9018
Epoch 56/200
110288/110288 [==============================] - 14s - loss: 0.3441 - acc: 0

110288/110288 [==============================] - 14s - loss: 0.2812 - acc: 0.9217 - val_loss: 0.3000 - val_acc: 0.9184
Epoch 79/200
110288/110288 [==============================] - 14s - loss: 0.2764 - acc: 0.9232 - val_loss: 0.3141 - val_acc: 0.9147
Epoch 80/200
110288/110288 [==============================] - 14s - loss: 0.2785 - acc: 0.9224 - val_loss: 0.3057 - val_acc: 0.9174
Epoch 81/200
110288/110288 [==============================] - 14s - loss: 0.2738 - acc: 0.9236 - val_loss: 0.3048 - val_acc: 0.9168
Epoch 82/200
110288/110288 [==============================] - 14s - loss: 0.2796 - acc: 0.9220 - val_loss: 0.3038 - val_acc: 0.9186
Epoch 83/200
110288/110288 [==============================] - 14s - loss: 0.2723 - acc: 0.9241 - val_loss: 0.2895 - val_acc: 0.9212
Epoch 84/200
110288/110288 [==============================] - 14s - loss: 0.2735 - acc: 0.9236 - val_loss: 0.2863 - val_acc: 0.9227
Epoch 85/200
110288/110288 [==============================] - 14s - loss: 0.2628 - acc: 0

110288/110288 [==============================] - 14s - loss: 0.1876 - acc: 0.9495 - val_loss: 0.2177 - val_acc: 0.9432
Epoch 138/200
110288/110288 [==============================] - 14s - loss: 0.1867 - acc: 0.9500 - val_loss: 0.2131 - val_acc: 0.9437
Epoch 139/200
110288/110288 [==============================] - 14s - loss: 0.1876 - acc: 0.9491 - val_loss: 0.2134 - val_acc: 0.9432
Epoch 140/200
110288/110288 [==============================] - 14s - loss: 0.1928 - acc: 0.9476 - val_loss: 0.2090 - val_acc: 0.9453
Epoch 141/200
110288/110288 [==============================] - 14s - loss: 0.1859 - acc: 0.9501 - val_loss: 0.2035 - val_acc: 0.9471
Epoch 142/200
110288/110288 [==============================] - 14s - loss: 0.1865 - acc: 0.9496 - val_loss: 0.2014 - val_acc: 0.9473
Epoch 143/200
110288/110288 [==============================] - 14s - loss: 0.1849 - acc: 0.9500 - val_loss: 0.2024 - val_acc: 0.9469
Epoch 144/200
110288/110288 [==============================] - 14s - loss: 0.1889 -

110288/110288 [==============================] - 14s - loss: 0.1568 - acc: 0.9581 - val_loss: 0.1740 - val_acc: 0.9554
Epoch 199/200
110288/110288 [==============================] - 14s - loss: 0.1539 - acc: 0.9589 - val_loss: 0.1796 - val_acc: 0.9541
Epoch 200/200
110288/110288 [==============================] - 14s - loss: 0.1563 - acc: 0.9580 - val_loss: 0.1933 - val_acc: 0.9494
Sample 1:
il a vu un vieux camion jaune <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Il a vu un vieux camion jaune
Sample 2:
new jersey est parfois calme pendant l' et il est est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


## Submission
When you are ready to submit your project, do the following steps:
1. Ensure you pass all points on the [rubric](https://review.udacity.com/#!/rubrics/1004/view).
2. Submit the following in a zip file.
  - `helper.py`
  - `machine_translation.ipynb`
  - `machine_translation.html`
    - You can export the notebook by navigating to **File -> Download as -> HTML (.html)**.